# Classification of Events in High Energy Physics: Signal or Background?

Event classification plays a crucial role in High Energy Physics (HEP). When analyzing experimental data, physicists often search for rare events hidden within a vast background of unwanted interactions. These backgrounds can arise for various reasons: the event of interest might be intrinsically rare, or the experimental setup itself may produce a high rate of background events.

Traditionally, physicists have tackled this problem by applying selection criteria to specific event features, such as kinematic variables. By identifying patterns in these variables, one can enhance the signal-to-background ratio, increasing the likelihood of detecting meaningful events. However, as experiments become more complex and datasets grow larger, machine learning (ML) methods offer a more powerful and automated approach to classification.

In this tutorial, we will introduce a simple ML-based approach to event classification. We will start from the basics, using a straightforward dataset and applying a classification model to separate signal from background.

# Implementation of an event classifier:

We want to classify each event of our dataseta as either:
- **Signal (S)**: interesting events, *to be identified*
- **Background (B)**: irrelevant events, *to be rejected*

This will be done in the following steps:
1. **Data Preprocessing**: import, clean, and normalize data
2. **Features Selection**: evaluate which variables have more discriminating power
3. **Choice of the Model**: pick an appropriate classifier
4. **Model Training**: train the model on the training set and tune parameters
5. **Model Evaluation**: Check accuracy, precision, ROC curve and score
6. **Predictions**: Use the trained model to predict classes for an independent dataset


### 1) Data Preprocessing

#### Import needed packages
*([Scikit learn documentation for classification problem](https://scikit-learn.org/stable/auto_examples/classification/index.html))*

In [ ]:
import uproot
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

#### Load and clean data

In [ ]:
signal_root_file = uproot.open("./reduced_signal_forTraining.root")
background_root_file = uproot.open("./reduced_background_forTraining.root")

In [ ]:
signal_tree = signal_root_file["zp"]
background_tree = background_root_file["zp"]

In [ ]:
raw_variables = ['Zp_cm_p_beforeFit', 'P_beforeFit', 'Zp_tau0_CMS_p_beforeFit', 'Zp_tau1_CMS_p_beforeFit', 
                 'fake_tau0_CMS_p_beforeFit', 'fake_tau1_CMS_p_beforeFit', 'fake_tau0_CMS_pt_beforeFit', 
                 'fake_tau1_CMS_pt_beforeFit', 'fake_PtTheta_beforeFit', 'fake_PtRho_beforeFit', 
                 'IntSymmZp_Pt_beforeFit', 'DistZp_Pt_beforeFit', 'Zp_PtTheta_sign_beforeFit', 
                 'Zp_PtRho_sign_beforeFit', 'Zp_ptPtMin_Theta_beforeFit', 'Zp_ptPtMin_Rho_beforeFit']
mcTruth_variables = ['Zp_tau_0_dau_genMotherPDG', 'Zp_tau_1_dau_genMotherPDG']
mass_variables = ['Zp_M_beforeFit']

In [ ]:
branches = raw_variables+mcTruth_variables+mass_variables

In [ ]:
print(branches)

In [ ]:
background_data = background_tree.arrays(branches, library="pd")
signal_data = signal_tree.arrays(branches, library="pd")

In [ ]:
background_data = background_data[(background_data['Zp_M_beforeFit'] > 4.5) & (background_data['Zp_M_beforeFit'] < 5.5)]                                 
signal_data = signal_data[(signal_data['Zp_tau_0_dau_genMotherPDG'] == 26) & (signal_data['Zp_tau_1_dau_genMotherPDG'] == 26)]

#### How is our dataset like?

In [ ]:
fig = plt.figure(figsize=(14, 10))
fig.suptitle('Mass Distribution', fontsize=22, y = 0.93)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.2], wspace=0.2)
ax1 = fig.add_subplot(gs[0])
ax1.hist([background_data['Zp_M_beforeFit'], signal_data['Zp_M_beforeFit']], range = (4.5, 5.5), bins=60, 
         color=['red', 'blue'], alpha=0.7, label=['Background', 'Signal'], edgecolor='none', density = True, 
         stacked=True)
ax1.set_xlabel('M(Z$^{\prime}$) [GeV/c$^{2}$]', fontsize = 15)
ax1.set_ylabel('Frequency', fontsize = 15)
ax1.legend(frameon = False, fontsize = 15)

gs_right = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[1], hspace=0)
ax2 = fig.add_subplot(gs_right[0])
ax2.hist(signal_data['Zp_M_beforeFit'], range = (4.5, 5.5), bins=60, color='blue', alpha=0.6, edgecolor='none', 
         density = True)
ax2.set_xlabel('M(Z$^{\prime}$) [GeV/c$^{2}$]', fontsize = 15)
ax2.set_ylabel('Frequency', fontsize = 15)

ax3 = fig.add_subplot(gs_right[1])
ax3.hist(background_data['Zp_M_beforeFit'], range = (4.5, 5.5), bins=60, color='red', alpha=0.6, edgecolor='none', 
         density = True)
ax3.set_xlabel('M(Z$^{\prime}$) [GeV/c$^{2}$]', fontsize = 15)
ax3.set_ylabel('Frequency', fontsize = 15)

plt.show()

### 2) Feature Selection

In [ ]:
fig = plt.figure(figsize=(15, 10))
fig.suptitle('A couple of Kinematics Distributions', fontsize=22, y = 0.93)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1.2], wspace=0.2)
ax1 = fig.add_subplot(gs[0])
ax1.hist(signal_data['Zp_cm_p_beforeFit'], bins=60, color='blue', alpha=0.6, label='Signal', 
           edgecolor='none', density = True)
ax1.hist(background_data['Zp_cm_p_beforeFit'], bins=60, color='red', alpha=0.6, label='Background', 
           edgecolor='none', density = True)
ax1.set_xlabel('P$_{CMS}$ (Z$^{\prime}$) [GeV/c]', fontsize=15)
ax1.set_ylabel('Density', fontsize=15)
ax1.legend(frameon = False, fontsize=15)

gs_right = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[1], hspace=0.2)
ax2 = fig.add_subplot(gs_right[0])
ax2.hist2d(signal_data['fake_tau0_CMS_pt_beforeFit'], signal_data['fake_tau1_CMS_pt_beforeFit'], 
             bins=(40, 40), cmap='Blues', density=True)
ax2.set_xlabel('P$^{t}_{CMS}$(R$^{\prime}_{\mu_{0}}$) [GeV]', fontsize=15)
ax2.set_ylabel('P$^{t}_{CMS}$(R$^{\prime}_{\mu_{1}}$) [GeV]', fontsize=15)
ax2.text(3.6, 3.8, "Signal", fontsize=15, color="black", ha="center", va="center")

ax3 = fig.add_subplot(gs_right[1])
ax3.hist2d(background_data['fake_tau0_CMS_pt_beforeFit'], background_data['fake_tau1_CMS_pt_beforeFit'], 
             bins=(40, 40), cmap='Reds', density=True)
ax3.set_xlabel('P$^{t}_{CMS}$(R$^{\prime}_{\mu_{0}}$) [GeV]', fontsize=15)
ax3.set_ylabel('P$^{t}_{CMS}$(R$^{\prime}_{\mu_{1}}$) [GeV]', fontsize=15)
ax3.text(3.6, 3.8, "Background", fontsize=15, color="black", ha="center", va="center")

plt.show()

In [ ]:
selected = ['Zp_cm_p_beforeFit', 'P_beforeFit', 'Zp_tau0_CMS_p_beforeFit', 'Zp_tau1_CMS_p_beforeFit', 
            'fake_tau0_CMS_p_beforeFit', 'fake_tau1_CMS_p_beforeFit', 'fake_tau0_CMS_pt_beforeFit', 
            'fake_tau1_CMS_pt_beforeFit', 'fake_PtTheta_beforeFit', 'fake_PtRho_beforeFit', 
            'IntSymmZp_Pt_beforeFit', 'DistZp_Pt_beforeFit', 'Zp_PtTheta_sign_beforeFit', 
            'Zp_PtRho_sign_beforeFit', 'Zp_ptPtMin_Theta_beforeFit', 'Zp_ptPtMin_Rho_beforeFit']

#### prepare dataset for the training: train and test fraction, scaling

In [ ]:
X_signal = signal_data[selected].values
X_background = background_data[selected].values

In [ ]:
y_signal = np.ones(len(signal_data)) 
y_background = np.zeros(len(background_data)) 

In [ ]:
X = np.vstack((X_signal, X_background))  
y = np.hstack((y_signal, y_background))  

In [ ]:
X, y = shuffle(X, y, random_state=42)

#### Use  `train_test_split` from *scikit learn* for determining the fraction of events that should be used for training and testing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Use `StandardScaler` from *scikit learn* for scaling the dataset 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### save the scaler for further implementation

In [ ]:
import joblib
joblib.dump(scaler, "scaler.pkl")

#### check features correlation with `train_df.corr()`

In [ ]:
train_df = pd.DataFrame(X_train, columns=selected)

In [ ]:
corr_matrix = train_df.corr()

In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu', fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix of Features", fontsize=22, y=1.02)
plt.show()

###  3) Choice of the Model

#### import classifiers and load the models you want to test for classification

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [ ]:
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=500),
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'MLP Neural Network': MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

#### train your models using `model.fit(train_sample, test_sample)`

In [ ]:
for name, model in classifiers.items():
    model.fit(X_train, y_train)

#### compute ROC curves and AUC scores for all classifiers to compare their performance

In [ ]:
roc_auc_scores = []

for name, model in classifiers.items():

    y_pred_prob = model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    roc_auc_scores.append((name, fpr, tpr, roc_auc))

In [ ]:
roc_auc_scores.sort(key=lambda x: x[3], reverse=True)

In [ ]:
plt.figure(figsize=(10, 8))
colors = plt.cm.plasma(np.linspace(0, 1, len(classifiers)))

for (name, fpr, tpr, roc_auc), color in zip(roc_auc_scores, colors):
    plt.plot(fpr, tpr, color=color, lw=1, label=f"{name} (AUC = {roc_auc:.3f})")

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")

plt.xlabel("False Positive Rate (Background Acceptance)", fontsize=15)
plt.ylabel("True Positive Rate (Signal Efficiency)", fontsize=15)
plt.title("ROC Curve Comparison", fontsize=22)
plt.legend(fontsize=15)
plt.grid()
plt.tick_params(axis="both", labelsize=15)
plt.show()

### 4) Model Training

We decided to use a Multi-Layer Perceptron for our classification problem. At this point we need to tune the parameters of the model, in order to optimize the performance and the robustness of the algorithm. 

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive

In [ ]:
# Function to update and plot ROC curve
def plot_roc(hidden_layer_1, hidden_layer_2, activation, solver, alpha, learning_rate, max_iter):
    hidden_layers = (hidden_layer_1, hidden_layer_2)
    
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layers,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate=learning_rate,
        max_iter=max_iter,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='navy', label=f'MLPClassifier (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("False Positive Rate", fontsize=15)
    plt.ylabel("True Positive Rate", fontsize=15)
    plt.title("ROC Curve - MLP Classifier", fontsize=22)
    plt.legend(loc="lower right", fontsize=15)
    plt.grid()
    plt.tick_params(axis="both", labelsize=15)
    plt.show()

In [ ]:
hidden_layer_1_widget = widgets.IntSlider(value=64, min=16, max=80, step=2, description='Layer 1 Neurons:')
hidden_layer_2_widget = widgets.IntSlider(value=32, min=16, max=80, step=2, description='Layer 2 Neurons:')
activation_widget = widgets.Dropdown(options=['identity', 'logistic', 'tanh', 'relu'], value='relu', description='Activation:')
solver_widget = widgets.Dropdown(options=['lbfgs', 'sgd', 'adam'], value='adam', description='Solver:')
alpha_widget = widgets.FloatLogSlider(value=0.0001, min=-5, max=0, step=0.1, base=10, description='Alpha:')
learning_rate_widget = widgets.Dropdown(options=['constant', 'invscaling', 'adaptive'], value='constant', description='Learning Rate:')
max_iter_widget = widgets.IntSlider(value=500, min=100, max=2000, step=100, description='Max Iterations:')

In [ ]:
display(widgets.interactive(plot_roc, 
                            hidden_layer_1=hidden_layer_1_widget,
                            hidden_layer_2=hidden_layer_2_widget,
                            activation=activation_widget,
                            solver=solver_widget,
                            alpha=alpha_widget,
                            learning_rate=learning_rate_widget,
                            max_iter=max_iter_widget))

*The adjustment of parameters can in turn be entrusted to automatic tools found in scikit leanr. Take a look [here](https://scikit-learn.org/stable/modules/grid_search.html) to learn more!*

#### train your model with the optimized parameters using `model.fit(X_train, y_train)`

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

#### save the model for later applications

In [ ]:
import joblib
joblib.dump(mlp, "mlp_model.pkl")

### 4) Model Evaluation

#### Make prediction on the test sample (30% of the total in our case) using `predict(x_test_sample)`

In [ ]:
y_pred = mlp.predict(X_test)

#### Predict class probability on the test sample using `predict_proba`

In [ ]:
y_prob = mlp.predict_proba(X_test)[:, 1]  

#### Confusion Matrix
*See documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)*

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.heatmap(cm, annot=True, fmt='d', cmap="Purples", linewidths=0.0, linecolor='black')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Predicted Label", fontsize=15)
plt.ylabel("True Label", fontsize=15)
plt.title("Confusion Matrix", fontsize=22)

#### Plot the ROC curve (`roc_curve`) and the AUC (`auc`) 
*See documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)*

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, label=f"MLP (AUC = {roc_auc:.2f})", color="Purple")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")  # Random guess line
plt.xlabel("False Positive Rate", fontsize=15)
plt.ylabel("True Positive Rate", fontsize=15)
plt.title("ROC Curve", fontsize=22)
plt.legend(frameon=False, fontsize=15)
plt.grid()
plt.tick_params(axis="both", labelsize=15)
plt.show()

#### Plot the probability (`y_prob`) for each class

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(y_prob[y_test == 1], bins=50, alpha=0.6, color='r', label='Signal')
plt.hist(y_prob[y_test == 0], bins=50, alpha=0.6, color='b', label='Background')
plt.xlabel("MLP Output Score", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.title("MLP Output Distribution (Signal vs Background)", fontsize=22)
plt.legend(frameon=False, fontsize=15)
plt.tick_params(axis="both", labelsize=15)
plt.yscale('log')
plt.show()

### 5) Predictions on an independent dataset

#### Load the model previously trained and evaluated

In [ ]:
mlp = joblib.load("mlp_model.pkl")  
scaler = joblib.load("scaler.pkl")  

#### Load the independent dataset and preprocess it as the one used for training
*OBS: the dataset must have an identical structure to the trained one*

In [ ]:
app_signal_root_file = uproot.open("./reduced_signal_forApplication.root")
app_background_root_file = uproot.open("./reduced_background_forApplication.root")

In [ ]:
app_signal_tree = app_signal_root_file["zp"]
app_background_tree = app_background_root_file["zp"]

In [ ]:
app_signal_data = app_signal_tree.arrays(branches, library="pd")
app_background_data = app_background_tree.arrays(branches, library="pd")

In [ ]:
app_signal_data = app_signal_data[(app_signal_data['Zp_tau_0_dau_genMotherPDG'] == 26) & (app_signal_data['Zp_tau_1_dau_genMotherPDG'] == 26)]
app_background_data = app_background_data[(app_background_data['Zp_M_beforeFit'] > 4.5) & (app_background_data['Zp_M_beforeFit'] < 5.5)]                                 

In [ ]:
app_X_signal = app_signal_data[selected].values
app_X_background = app_background_data[selected].values

In [ ]:
app_X_signal_new = scaler.transform(app_X_signal)
app_X_background_new = scaler.transform(app_X_background)

#### Make prediction and compute the probability of each class

In [ ]:
signal_predictions = mlp.predict(app_X_signal_new)
background_predictions = mlp.predict(app_X_background_new)

In [ ]:
signal_probs = mlp.predict_proba(app_X_signal_new)[:, 1]
background_probs = mlp.predict_proba(app_X_background_new)[:, 1]

#### Prepare a dataset only including a variable on which we want to evaluate the background rejection vs. the signal efficiency

In [ ]:
mass_signal = app_signal_data["Zp_M_beforeFit"]
mass_background = app_background_data["Zp_M_beforeFit"]

In [ ]:
def plot_interactive_cut(threshold):
    # Plot MLP score distribution
    plt.figure(figsize=(12, 6))

    # Create a subplot for MLP Scores
    plt.subplot(1, 2, 1)
    plt.hist(signal_probs, bins=50, range=(0, 1), alpha=0.6, label="Signal", color="red", histtype="stepfilled", density=True)
    plt.hist(background_probs, bins=50, range=(0, 1), alpha=0.6, label="Background", color="blue", histtype="stepfilled", density=True)
    plt.axvline(threshold, color="red", linestyle="--", linewidth=1, label=f"Cut @ {threshold:.2f}")
    plt.yscale('log')
    plt.xlabel("MLP Score", fontsize=15)
    plt.ylabel("Normalized Events", fontsize=15)
    plt.title("MLP Output Scores", fontsize=22)
    plt.legend(frameon=False, fontsize=10)
   

    # Apply the threshold cut for signal and background
    signal_pass = signal_probs > threshold
    background_pass = background_probs > threshold

    # Get the mass distributions for events that pass the cut
    mass_signal_cut = mass_signal[signal_pass]
    mass_background_cut = mass_background[background_pass]
    
    # Signal Efficiency (percentage of signal events passing the threshold)
    signal_efficiency = len(mass_signal_cut) / len(mass_signal) if len(mass_signal) > 0 else 0

    # Background Rejection (percentage of background events *not* passing the threshold)
    background_rejection = 1 - len(mass_background_cut) / len(mass_background) if len(mass_background) > 0 else 0

    # Sensitivity calculation (simplified: signal efficiency / sqrt(background events))
    sensitivity = signal_efficiency / np.sqrt(len(mass_background_cut) + 1e-10) 
    # Create a subplot for Mass Distribution
    plt.subplot(1, 2, 2)
    plt.hist([mass_background_cut, mass_signal_cut], range=(4.5, 5.5), bins=50, stacked=True, label=["Background (after cut)", "Signal (after cut)"], 
             color=["blue", "red"], alpha=0.6)
    plt.xlabel("Mass [GeV]", fontsize=15)
    plt.ylabel("Frequency", fontsize=15)
    plt.title("Mass Distribution After MLP Cut", fontsize=22)
    plt.legend(frameon=False, fontsize=10)


    # Add sensitivity as text on the plot
    plt.figtext(0.813, 0.76, f"Sensitivity: {sensitivity:.3f}\nSignal Efficiency: {signal_efficiency*100:.1f}%\nBkg Rejection: {background_rejection*100:.1f}%",
                ha='left', va='center', fontsize=12, 
                bbox=dict(facecolor='white', alpha=0, edgecolor='none', boxstyle='square,pad=1'))
    plt.tight_layout()

    plt.show()

# Create a slider for the threshold (range from 0 to 1, default at 0.5)
threshold_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Threshold:', continuous_update=False)

# Use interactive to update the plot when the slider is moved
interactive_plot = interactive(plot_interactive_cut, threshold=threshold_slider)

# Display the interactive plot
interactive_plot